In [1]:
import numpy as np
from scipy.io import loadmat
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
hf = h5py.File('/media/maria/DATA1/Documents/ZebraFish/subject_1/TimeSeries.h5', 'r')
#print(hf['absIx'])
neural_dat=np.array(hf['CellResp'])
del hf

In [3]:
regressors=loadmat('/media/maria/DATA1/Documents/ZebraFish/subject_1/data_full.mat')
cell_pos=regressors['data']['CellXYZ'][0][0]
missing_values=regressors['data']['IX_inval_anat'][0][0]
cell_pos=np.delete(cell_pos,missing_values,axis=0)

In [4]:
print(cell_pos)

[[1163 1013    1]
 [1140 1009    1]
 [1102 1007    1]
 ...
 [1044  276   29]
 [ 948  271   29]
 [1018  268   29]]


In [20]:
def circle(center_x,center_y,radius,h,k):
    within_circle=radius>np.sqrt((center_x-h)**2+(center_y-k)**2)
    #print(within_circle)
    return within_circle

In [23]:
center_x=cell_pos[100][0]
center_y=cell_pos[100][0]

boolean_arr=[]
for j in range(0,83205):
    boolean_ind=circle(center_x,center_y,100,cell_pos[j][0],cell_pos[j][1])
    boolean_arr.append(boolean_ind)
print(sum(boolean_arr))

/home/maria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in ushort_scalars
  


162


In [15]:
print(cell_pos.shape)

(83205, 3)


In [24]:
selected_cells=neural_dat[:,boolean_arr]

In [ ]:
plt.plot(np.mean(sele))